# Looking at waveforms using cuts on other tiers

Use the legend-sw kernel<br>

Jita (06 Feb 2023)

In [1]:
# Set up python environment
from pygama.raw.build_raw import build_raw
from pygama.dsp import build_dsp
#from pygama.lgdo import ls
import pygama.lgdo.lh5_store as lh5 
from pygama.vis.waveform_browser import WaveformBrowser

hit_file = "/global/cscratch1/sd/jita/LGND-Data/r010/hit/l200-p02-r010-cal-20230126T023019Z-tier_hit.lh5"
raw_file = "/global/cscratch1/sd/jita/LGND-Data/r010/raw/l200-p02-r010-cal-20230126T023019Z-tier_raw.lh5"
dsp_file = "/global/cscratch1/sd/jita/LGND-Data/r010/dsp/l200-p02-r010-cal-20230126T023019Z-tier_dsp.lh5"

## Inspecting the dsp file

In [2]:
raw_df = lh5.load_dfs(raw_file, ['baseline','daqenergy','eventnumber','packet_id','runtime'], 'ch004/raw')
print(raw_df)
dsp_df = lh5.load_dfs(dsp_file, ['baseline','bl_intercept','bl_mean','bl_slope','bl_std','cuspEmax','timestamp','trapEmax','zacEmax'], 'ch004/dsp')
print(dsp_df)
hit_df = lh5.load_dfs(hit_file, ['AoE_Classifier','AoE_Corrected','AoE_Double_Sided_Cut','AoE_Low_Cut','cuspEftp_ctc_cal','cuspEmax_ctc_cal','is_negative','is_saturated','is_valid_0vbb','is_valid_cal','timestamp','trapEmax_ctc_cal','trapTmax_cal','zacEftp_ctc_cal','zacEmax_ctc_cal'], 'ch004/hit')
print(hit_df)

      baseline  daqenergy  eventnumber  packet_id     runtime
0        14989       1928           24        310    0.470211
1        15006       2598           49        616    0.522526
2        15016       3076           72        896    0.573396
3        14989      19450           77        952    0.587104
4        15143      11898           78        966    0.589458
...        ...        ...          ...        ...         ...
3538     15020       1364        79913     976664  175.831585
3539     14990       1189        79940     976996  175.893094
3540     15006      15766        79989     977586  176.000608
3541     15007       1229        80001     977732  176.021772
3542     15028       1163        80005     977790  176.030559

[3543 rows x 5 columns]
      baseline  bl_intercept    bl_mean  bl_slope     bl_std      cuspEmax  \
0        14989      5.851315   6.945365  0.004726  12.829613   1343.910645   
1        15006     -1.179030   4.102017  0.022812  14.463848   1812.636719 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
cuspEmax_ctc_cal = hit_df['cuspEmax_ctc_cal']
energy_selection = (cuspEmax_ctc_cal >= 2610) & (cuspEmax_ctc_cal < 2620)
cuspEmax_ctc_cal.plot.hist(grid=False, bins=1000, range=(0, 3000),rwidth=1.,color='coral')
cuspEmax_ctc_cal[energy_selection].plot.hist(grid=False, bins=1000, range=(0, 3000),rwidth=1.,color='blue')
plt.yscale('log')

In [ ]:
# Now construct a WaveformBrowser with this cut
browser = WaveformBrowser(raw_file,
                          'ch004/raw',
                          aux_values=hit_df,
                          legend="E = {cuspEmax_ctc_cal}",
                          entry_mask=energy_selection,
                          n_drawn =5,
                          y_lim=(10000,50000),
                          x_lim=(47000,50000))

In [ ]:
# Draw the next 3 batches of 5 waveforms
for entries, i in zip(browser, range(2)):
    print("Entries:", entries)
    browser.new_figure()